# Preprocessing of clinical data

Preprocessing of the clinical data from the EHR (electronic health record).

## Data import

In [1]:
import pandas as pd

df = pd.read_csv("../../data/raw/ResectMap_DATA_2023-11-13_2306.csv")

# Cell to set the flag variable
execute_special_cell = False  # Set this to True to run the special cell

C:\Users\Amaury\AppData\Local\Temp\ipykernel_28428\79705969.py:3: DtypeWarning: Columns (8,20,31,39,65,79,83,85,127,130,132,150,153,161,308,313,317,321,325,329,330,334,377,385,403,409,439,440,441,442,443,444,445,453,463,475,512,523,524,525,526,527,528,529,537,566,572,574,588,589,590,591,592,601,602,604,605,606,608,620,621,622,627,633,636,637,639,646,647,658,676,677,682,695,698,712,714,718,720,721,723,725,726,728,732,733,735,748,750,754,757,759,761,762,764,768,770,772,774,776,780,782,786,787,791,796,803,810,812,813,814,815,817,818,880,887,888,896,904,905,906,968,975,976,984,992,993,994,1056,1063,1064,1072,1080,1081,1082,1144,1151,1152,1169,1170,1232,1239,1240,1257,1258,1320,1327,1328,1345,1408,1415,1416,1432,1433,1496,1503,1521,1523,1540,1541,1558,1559,1576,1577,1595,1613,1631,1649,1667,1669,1670,1671,1678,1730,1738,1790,1801,1802,1811,1819,1820,1821,1822,1823,1830,1882,1890,1942,1953,1954,1963,1971,1972,1973,1974,1975,1982,2034,2042,2094,2105,2106,2115,2124,2125,2126,2127,2134,2186,219

## "Most Recent" implementation

In [2]:
# Convert date of EO assessment as datetime format
df['eo_date_clin'] = pd.to_datetime(df['eo_date_clin'], format='%m/%d/%Y', errors='coerce')

# Only keep the most recent engel outcome assessment per 'record_id'
engel_outcomes = df[df['redcap_repeat_instrument'] == 'engel_outcomes'][['record_id', 'eo_date_clin']]
engel_outcomes = engel_outcomes.sort_values(by='eo_date_clin', ascending=False).drop_duplicates(subset='record_id', keep='first')

# Merge this subset with the original dataframe on 'record_id' 
df = df.merge(engel_outcomes, on='record_id', suffixes=('', '_engel'), how='left')

# Use transform to broadcast 'eo_date_surg' values to all rows per 'record_id'
#df['eo_date_clin'] = df.groupby('record_id')['eo_date_clin'].transform('first')

# Define the columns for each 'redcap_repeat_instrument' type
date_columns = {
    'surgical_information': 'resect_date',
    'eeg': 'eeg_date',
    'neuropsychological_testing': 'np_eval_date',
    'mri': ['doi', 'doi2']  # Include both 'doi' and 'doi2' for 'mri'
}

In [3]:
# Convert date of EO assessment as datetime format
#df['eo_date_surg_engel'] = pd.to_datetime(df['eo_date_surg_engel'], format='%m/%d/%Y', errors='coerce')

for instrument, date_columns in date_columns.items():
    # Create a subset of 'df' for the specific 'redcap_repeat_instrument' type
    subset_info = df[df['redcap_repeat_instrument'] == instrument].copy()

    # Convert the date column(s) from string format to datetime objects
    if isinstance(date_columns, list):  # Check if there are multiple date columns (e.g., for 'mri')
        for col in date_columns:
            subset_info[col] = pd.to_datetime(subset_info[col], format='%m/%d/%Y', errors='coerce')
        # If 'doi' is NaT (not a time), use 'doi2'
        subset_info['final_date'] = subset_info[date_columns[0]].fillna(subset_info[date_columns[1]])
    else:
        subset_info[date_columns] = pd.to_datetime(subset_info[date_columns], format='%m/%d/%Y', errors='coerce')
        subset_info['final_date'] = subset_info[date_columns]

    # Calculate the absolute difference in days between the type-specific date and the date of EO assessment
    subset_info['date_diff_abs'] = (subset_info['final_date'] - subset_info['eo_date_clin_engel']).dt.days.abs()
    subset_info['date_diff'] = (subset_info['final_date'] - subset_info['eo_date_clin_engel']).dt.days

    # Check if there are multiple rows for each 'record_id'
    group_counts = subset_info.groupby('record_id')['date_diff'].transform('size')
    multiple_rows_mask = group_counts > 1

    # Rank the 'date_diff' within each 'record_id' group and assign the rank as 'new_instance' for groups with more than one row
    #subset_info.loc[multiple_rows_mask, 'new_instance'] = subset_info[multiple_rows_mask].groupby('record_id')['date_diff'].rank(method='first')
    subset_info.loc[multiple_rows_mask, 'new_instance'] = subset_info[multiple_rows_mask].groupby('record_id')['final_date'].rank(ascending=False)
    
    # For groups with only one row, set 'new_instance' to 1
    subset_info.loc[~multiple_rows_mask, 'new_instance'] = 1

    # Update 'redcap_repeat_instance' in the main dataframe
    df.loc[subset_info.index, 'redcap_repeat_instance'] = subset_info['new_instance']
    df.loc[subset_info.index, 'date_diff'] = subset_info['date_diff']
    
# Cleaning up
#df.drop(columns=['eo_date_clin_engel', 'date_diff'], inplace=True, errors='ignore')

df.to_csv("../../data/processed/test.csv")

## Select only the most relevant information per patient

To do so, we select the last surgical that is neither a shunt or implementing a stimulation device and use this date as reference.

Afterwards, we select the eeg, neuropsych and mri examinations that happened closest to this last date of surgery reference.

### Find date of last surgery for each patient

In [ ]:
# Get ID and resect date
surgery_dates = df.loc[df["redcap_repeat_instrument"]=="surgical_information"]#[["record_id", "resect_date"]]

# Transform date column into datetime type/format
surgery_dates["resect_date"] = pd.to_datetime(surgery_dates["resect_date"], format='%m/%d/%Y', errors='coerce')

# Specify columns to check for the condition (no shunt or stimulation)
columns_to_check = [
    "resect_procedure___2", "resect_procedure___3", "resect_procedure___4", 
    "resect_procedure___5", "resect_procedure___6", "resect_procedure___7", 
    "resect_procedure___8", "resect_procedure___9", "resect_procedure___11", 
    "resect_procedure___12", "resect_procedure___13"
]

# Remove shunt from being considered
surgery_dates_no_shunt = surgery_dates.loc[df[columns_to_check].any(axis=1)]

# Get most recent date within subset
date_of_last_surgery = surgery_dates_no_shunt.groupby("record_id")["resect_date"].max().reset_index()
date_of_last_surgery = date_of_last_surgery.rename(columns={"resect_date" : "last_surg_date"})

# Check which participants don't have a resect date or only have shunts/stimulation
missing_dates = set(df['record_id']) - set(date_of_last_surgery['record_id'])

### Find eeg, neuropsych and mri closest to date of last surgery

In [ ]:
# Filter out the rows with 'engel_outcomes', sort by 'eo_date_surg' descending, and keep only the most recent occurrence per 'record_id'
engel_outcomes = df[df['redcap_repeat_instrument'] == 'engel_outcomes'][['record_id', 'eo_date_surg']]
engel_outcomes = engel_outcomes.sort_values(by='eo_date_surg', ascending=False).drop_duplicates(subset='record_id', keep='first')

# Merge this subset with the original dataframe on 'record_id' 
df = df.merge(engel_outcomes, on='record_id', suffixes=('', '_engel'), how='left')

# Use transform to broadcast 'eo_date_surg' values to all rows per 'record_id'
df['eo_date_surg'] = df.groupby('record_id')['eo_date_surg'].transform('first')

# Define the columns for each 'redcap_repeat_instrument' type
date_columns = {
    'surgical_information': 'resect_date',
    'eeg': 'eeg_date',
    'neuropsychological_testing': 'np_eval_date',
    'mri': ['doi', 'doi2']  # Include both 'doi' and 'doi2' for 'mri'
}

for instrument, date_columns in date_columns.items():
    # Create a subset of 'df' for the specific 'redcap_repeat_instrument' type
    subset_info = df[df['redcap_repeat_instrument'] == instrument].copy()

    # Convert the date column(s) from string format to datetime objects
    if isinstance(date_columns, list):  # Check if there are multiple date columns (e.g., for 'mri')
        for col in date_columns:
            subset_info[col] = pd.to_datetime(subset_info[col], format='%m/%d/%Y', errors='coerce')
        # If 'doi' is NaT (not a time), use 'doi2'
        subset_info['final_date'] = subset_info[date_columns[0]].fillna(subset_info[date_columns[1]])
    else:
        subset_info[date_columns] = pd.to_datetime(subset_info[date_columns], format='%m/%d/%Y', errors='coerce')
        subset_info['final_date'] = subset_info[date_columns]

    subset_info['eo_date_surg'] = pd.to_datetime(subset_info['eo_date_surg'], format='%m/%d/%Y', errors='coerce')

    # Calculate the absolute difference in days between the type-specific date and 'eo_date_surg'
    subset_info['date_diff'] = (subset_info['final_date'] - subset_info['eo_date_surg']).dt.days.abs()

    # Check if there are multiple rows for each 'record_id'
    group_counts = subset_info.groupby('record_id')['date_diff'].transform('size')
    multiple_rows_mask = group_counts > 1

    # Rank the 'date_diff' within each 'record_id' group and assign the rank as 'new_instance' for groups with more than one row
    subset_info.loc[multiple_rows_mask, 'new_instance'] = subset_info[multiple_rows_mask].groupby('record_id')['date_diff'].rank(method='first')

    # For groups with only one row, set 'new_instance' to 1
    subset_info.loc[~multiple_rows_mask, 'new_instance'] = 1

    # Update 'redcap_repeat_instance' in the main dataframe
    df.loc[subset_info.index, 'redcap_repeat_instance'] = subset_info['new_instance']
    
# Cleaning up
df.drop(columns=['eo_date_surg_engel', 'date_diff'], inplace=True, errors='ignore')

df.to_csv("../../data/processed/test.csv")

## Filter the data

1. Filter the dataframe for only the most recent information (redcap_repeat_instance == 1)
2. Check that there is no overlapping information per participant

In [ ]:
# Fill 'redcap_repeat_instance' with 1 only for rows where 'mrn' has a value
df.loc[df['mrn'].notna(), 'redcap_repeat_instance'] = df.loc[df['mrn'].notna(), 'redcap_repeat_instance'].fillna(1)

# Only keep rows where with only 1 measurement per instrument
df_subset = df[df["redcap_repeat_instance"]==1]

columns_to_exclude = ['record_id', 'redcap_repeat_instrument', 'redcap_repeat_instance', "eo_date_surg"]

# Get the list of columns to check by excluding the columns_to_exclude
columns_to_check = [col for col in df_subset.columns if col not in columns_to_exclude]

# Group by 'record_id' and check which columns have more than 1 row of information for each participant
check_result = df_subset.groupby('record_id')[columns_to_check].apply(lambda x: x.columns[x.notnull().sum() > 1].tolist())

# Print the result for each participant
for record_id, problematic_columns in check_result.items():
    if problematic_columns:
        print(f"Participant {record_id} has more than 1 row of information in columns: {', '.join(problematic_columns)}")

df_subset.to_csv("../../data/processed/test2.csv")
del columns_to_exclude, columns_to_check, check_result, record_id, problematic_columns

## Flatten the dataframe

Flatten the dataframe from a long dataframe to a wide dataframe with only 1 particpant per row.

In [ ]:
# Conditional cell execution
if execute_special_cell==False:
    
    # Group by 'record_id' and aggregate non-null values for each column
    wide_df = df_subset.groupby('record_id').agg(lambda x: x.dropna().iloc[0] if not x.dropna().empty else None)

    # Reset the index to get a clean DataFrame
    wide_df = wide_df.reset_index()

    # Now, consolidated_df contains one row per patient with non-null information
    print(wide_df)

    # Save dataframe for further use
    wide_df.to_csv("../../data/processed/wide_df.csv")

else:
    # Read dataframe from the pickle format
    wide_df = pd.read_csv("../../data/processed/wide_df.csv")

## Check for wrong datatypes

In [ ]:
wide_df.info(verbose=True, show_counts=True)

## Remove unnecessary columns

In [ ]:
columns_to_exclude = ["redcap_repeat_instrument", "redcap_repeat_instance", "mrn", "study_site_other", "ny_num_yn", "ny_num"]

###### IN PROGRESS ######

wide_df = wide_df.drop(columns_to_exclude, axis=1)

del columns_to_exclude

## Check for NAs

In [ ]:
# Set threshold (e.g., 0.5 for 50%)
threshold = 0.9

# Calculate the NaN percentage for each column
nan_percentages = wide_df.isna().mean()

# Count the columns where the NaN percentage exceeds the threshold
columns_above_threshold = nan_percentages[nan_percentages > threshold].index.tolist()

# Get the count of columns above the threshold
count_above_threshold = len(columns_above_threshold)

print(f"Number of columns with NaN percentage above {threshold * 100}%: {count_above_threshold}")

del threshold, nan_percentages, count_above_threshold

### Delete columns with NA % above threshold

In [ ]:
#wide_df = wide_df.drop(columns_above_threshold, axis=1)

## Add new features

### Number of measurements

Because the data is filtered for only "redcap_repeat_instance" == 1, we lose the information on other timepoints.
In order to capture that information, new variables/columns were created. These columns sum the number of measurement of each type per patient.

In [ ]:
# Pivot the dataframe
pivoted_df = df.pivot_table(index='record_id', columns='redcap_repeat_instrument', values='redcap_repeat_instance', aggfunc='count')

# Fill NaN values with 0
pivoted_df.fillna(0, inplace=True)

# Display the resulting dataframe
print(pivoted_df.head())

# Merge the dataframes based on the 'record_id' column
wide_df = pd.merge(wide_df, pivoted_df, on='record_id', how='left')

del pivoted_df

### Age of seizure onset

We extract the age of seizure onset from a text column ("seizure_class_notes)

In [ ]:
import re

def extract_age_of_onset(text):
    age_pattern = re.compile(r'''
        (?:age\s*of\s*seizure\s*onset|age\s*at\s*seizure\s*onset|age\s*of\s*onset|
         seizure\s*onset\s*age|seizure\s*age\s*of\s*onset|seizure\s*onset|
         seizures*\s*began\s*(?:at|at\s*the\s*age\s*of)*|(?:1st|first)\s*seizure|
         onset\s*at\s*age|age\s*(?=:)|seizure\s*onset\s*:\s*age)\s*:*\s*~*\s*
        (([0-9,.]+(?:\s*(?:-|to|or)\s*[0-9,.]+)?\s*(?:y/o|years?|months?|days?|weeks?))|
         in-utero|birth|(\d+\s*\+\s*\d+\s*months?)|\d+|(?:\d+\s*(?:years?|yrs)\s*\d+\s*months)|(?<=\bage\s)\d+)  # age with optional units or range
    ''', re.IGNORECASE | re.VERBOSE)

    match = re.search(age_pattern, text)
    if match:
        age_with_units = match.group(1)
        return convert_units_to_years(age_with_units)

    # Handle specific phrases
    if 'first seizure at birth' in text.lower() or 'seizures began at birth' in text.lower():
        return 0

    return None


def convert_units_to_years(age_with_units):
    # Handle special cases for birth and in-utero
    if 'birth' in age_with_units.lower():
        return 0
    if 'in-utero' in age_with_units.lower():
        return 0

    # Handle combined years and months (e.g., "43yrs 11months")
    combined_pattern = re.match(r'(\d+)\s*(?:years?|yrs)\s*(\d+)\s*months?', age_with_units, re.IGNORECASE)
    if combined_pattern:
        years, months = map(int, combined_pattern.groups())
        return years + months / 12

    # Handle range of ages
    if '-' in age_with_units or 'to' in age_with_units or 'or' in age_with_units:
        numbers = [float(n) for n in re.findall(r'\d+(?:\.\d+)?', age_with_units)]
        if numbers:
            average_age = sum(numbers) / len(numbers)
            return average_age / 12 if 'month' in age_with_units.lower() else average_age
        else:
            return None

    # Regular age extraction
    match = re.match(r'(\d+(?:[.,]\d+)?)\s*(y/o|years?|months?|days?|weeks?|birth?|in-utero?)?', age_with_units, re.IGNORECASE)
    
    if match:
        value, unit = match.groups()
        if value:
            value = float(value)
            if unit:
                if 'month' in unit.lower():
                    return value / 12
                elif 'week' in unit.lower():
                    return value / 52.1775
                elif 'day' in unit.lower():
                    return value / 365.25
                else:  # Assume it's years if no unit is specified
                    return value
            else:  # No unit specified, assume years
                return value
        else:
            return None

    return None

In [ ]:
# Apply the function to the 'doctor_notes' column and create a new 'age_of_onset' column
wide_df['age_of_onset'] = wide_df['seizure_class_notes'].apply(lambda x: extract_age_of_onset(str(x)) if pd.notna(x) else None)

result = wide_df[["age_of_onset", "seizure_class_notes"]]
result

This is how many "age at seizure onset" the RE was able to extract

In [ ]:
len(wide_df) - wide_df['age_of_onset'].isna().sum()

This is how many "age at seizure onset" after manually checking the dataseet

In [ ]:
# Conditional cell execution
if execute_special_cell:

    age_of_onset = wide_df[["record_id", "age_of_onset", "seizure_class_notes"]]
    age_of_onset = age_of_onset.rename(columns={"age_of_onset": "age_of_onset_hw"})
    age_of_onset.to_csv("../../data/processed/age_of_onset_hw.csv")
    
else:
    # Read the hand-written DataFrame
    age_of_onset_df = pd.read_csv("../../data/processed/age_of_onset_hw_done.csv")

    # Merge the hand-written data with wide_df
    wide_df = pd.merge(wide_df, age_of_onset_df[['record_id', 'age_of_onset_hw']], on='record_id', how='left')

    print(len(wide_df) - wide_df['age_of_onset_hw'].isna().sum())

### Clinical notes

Extract information from clinical notes such as the number of words.

In [ ]:
# Remove non special characters and lower the case
wide_df["seizure_class_notes"] = wide_df["seizure_class_notes"].str.replace('[^a-zA-Z]', ' ').str.lower()

# Put the number of words in seizure_notes_word_cnt
wide_df["seizure_notes_word_cnt"] = wide_df["seizure_class_notes"].str.split().str.len()

print(wide_df["seizure_notes_word_cnt"].head())

### Seizure frequency

We extract the seizure frequency from a text column ("seizure_freq) and convert it into a single unit (seizures per month).

In [ ]:
import pandas as pd
import re

def convert_to_monthly(freq_str, age):
    # Handling the case when the value is just '0'
    if freq_str.strip() == '0':
        return 0

    # Regular expression to extract numbers and unit, considering various formats
    match = re.match(r'(\d*\.?\d+)(?:\s*-\s*(\d*\.?\d+))?\s*(seizures?\/|seizures?\s+per\s+|per\s+|\/|\s+)?\s*(\w+)', freq_str, re.IGNORECASE)
    if match:
        num1, num2, _, unit = match.groups()
        num1 = float(num1)
        num2 = float(num2) if num2 else num1  # If no range, use the single number

        # Calculate the mean if there's a range
        number = (num1 + num2) / 2

        # Conversion rates to monthly frequency
        if unit in ['month', 'monthly', 'mo']:
            return number
        elif unit in ['day', 'daily']:
            return number * 30  # Approximate days in a month
        elif unit in ['week', 'weekly']:
            return number * 4.345  # Average weeks in a month
        elif unit in ['year', 'yearly']:
            return number / 12  # Months in a year
        elif unit == 'lifetime':
            if age > 0:
                # Convert lifetime frequency to monthly based on age
                return number / (age * 12)
            else:
                return None
        else:
            # Unknown unit
            return None
    else:
        # Pattern not matched
        return None

In [ ]:
# Apply the conversion to each row and create a new column
wide_df['freq_per_month'] = wide_df.apply(lambda row: convert_to_monthly(row['seizure_freq'], row['age']), axis=1)

result = wide_df[["freq_per_month", "seizure_freq"]]
result

This is how many seizure frequency the RE was able to extract.

In [ ]:
len(wide_df) - wide_df['freq_per_month'].isna().sum()

This is how many seizure frequency after manually checking.

In [ ]:
# Conditional cell execution
if execute_special_cell:

    seizure_freq = wide_df[["record_id", "freq_per_month", "seizure_freq"]]
    seizure_freq = seizure_freq.rename(columns={"freq_per_month": "freq_per_month_hw"})
    seizure_freq.to_csv("../../data/processed/freq_per_month_hw.csv")
    
else:
    # Read the hand-written DataFrame
    seizure_freq_df = pd.read_csv("../../data/processed/freq_per_month_hw_done.csv")

    # Merge the hand-written data with wide_df
    wide_df = pd.merge(wide_df, seizure_freq_df[['record_id', 'freq_per_month_hw']], on='record_id', how='left')

    print(len(wide_df) - wide_df['freq_per_month_hw'].isna().sum())

### Date of last surgery

As technology changes over time, so will surgery outcome.
We extract the year of last surgery.

In [ ]:
# Split the 'date' column and extract the year
wide_df['eo_year_surg'] = wide_df['eo_date_surg'].str.split('/').str[2]

# Convert the 'year' column to numeric (optional, if needed)
wide_df['eo_year_surg'] = pd.to_numeric(wide_df['eo_year_surg'])

wide_df['eo_year_surg'].head()

## Remapping sex

In [ ]:
# Before, 1 represented female and 2 represented male
# Now, 0 represents female and 1 represents male
wide_df["sex_gender"] = wide_df["sex_gender"].map({1: 0, 2: 1})

## Check for redundant columns

### Check for high correlation variables

In [ ]:
# Calculate the correlation matrix
correlation_matrix = wide_df.select_dtypes(exclude=['object']).corr()

# Set a threshold for high correlation (e.g., 0.8 for 80%)
threshold = 0.8

# Find pairs of variables with a very high correlation
high_correlation_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i + 1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            high_correlation_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j]))

# Print the pairs of variables with a very high correlation
print(f"Pairs of variables with correlation above {threshold}:")
for pair in high_correlation_pairs:
    print(pair)
    
del correlation_matrix, threshold, high_correlation_pairs, pair, i, j

### Check for low variance

In [ ]:
# Calculate the variance for each column
column_variances = wide_df.select_dtypes(exclude=['object']).var()

# Set a threshold for low variance
threshold = 0.1

# Find columns with a variance very close to 0
low_variance_columns = column_variances[column_variances < threshold]

# Print the count of columns with low variance and their names
count_low_variance_columns = len(low_variance_columns)
print(f"Number of columns with variance below {threshold}: {count_low_variance_columns}\n")

# Print the column names and their variances
print(f"Columns with variance below {threshold} (sorted by variance):")
for column, variance in low_variance_columns.sort_values().items():
    print(f"{column}: {variance}")
    
del threshold, low_variance_columns, count_low_variance_columns, column, variance

In [ ]:
# Count columns with a variance of 0
zero_variance_columns_count = (column_variances == 0).sum()

# Get the column names with a variance of 0
zero_variance_columns = column_variances[column_variances == 0].index.tolist()

# Print the count and names of columns with a variance of 0
print(f"Number of columns with a variance of 0: {zero_variance_columns_count}")
print(f"Columns with a variance of 0:")
for column in zero_variance_columns:
    print(column)
    
del zero_variance_columns_count, column_variances, column

### Delete columns with a variance of 0

In [ ]:
#wide_df = wide_df.drop(zero_variance_columns, axis=1)

del zero_variance_columns

### Delete redundant columns

## Creating a single outcome variable

In [ ]:
# Count rows where all surg_engel___1, 2, 3, 4 variables are 0
count_zero_rows = (wide_df[['surg_engel___1', 'surg_engel___2', 'surg_engel___3', 'surg_engel___4']] == 0).all(axis=1).sum()

# Display the count of rows with all 0 values
print("\nCount of rows with all 0 values:", count_zero_rows)

## Remove rows with no engel outcomes (all columns with 0 values)

In [ ]:
# Remove rows where all values are 0
wide_df = wide_df[~(wide_df[['surg_engel___1', 'surg_engel___2', 'surg_engel___3', 'surg_engel___4']] == 0).all(axis=1)]

# Count rows where all surg_engel___1, 2, 3, 4 variables are 0
count_zero_rows = (wide_df[['surg_engel___1', 'surg_engel___2', 'surg_engel___3', 'surg_engel___4']] == 0).all(axis=1).sum()

# Display the count of rows with all 0 values
print("\nCount of rows with all 0 values:", count_zero_rows)

In [ ]:
# Create 1 column out of the 4 dummy variables for engel outcome
wide_df['surg_engel'] = wide_df[['surg_engel___1', 'surg_engel___2', 'surg_engel___3', 'surg_engel___4']].idxmax(axis=1).str.split('___').str[1].astype(int)
wide_df[["record_id", "surg_engel",'surg_engel___1', 'surg_engel___2', 'surg_engel___3', 'surg_engel___4']].head()

In [ ]:
# Save dataframe for further use
wide_df.to_csv("../../data/processed/preprocessed_df.csv")

## Export engel outcomes and record_id

In [ ]:
engel_label_df = wide_df[['record_id', 'surg_engel']]

# Save dataframe for further use (i.e. MRI labeling)
engel_label_df.to_csv("../../data/processed/label_df.csv")

## Print system information

In [ ]:
import session_info

session_info.show()